In [2]:
import pandas as pd 
import numpy as np 

In [3]:
data = pd.read_csv('sqlResult.csv',encoding = 'gb18030')

In [4]:
data.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       89611 non-null  int64 
 1   author   79396 non-null  object
 2   source   89609 non-null  object
 3   content  87054 non-null  object
 4   feature  89611 non-null  object
 5   title    89577 non-null  object
 6   url      87144 non-null  object
dtypes: int64(1), object(6)
memory usage: 4.8+ MB


## 数据清洗

### 1 content和source为空的条目对于抄袭检测无意义

In [6]:
data = data.dropna(subset = ['source','content'],how = 'any')
data.reset_index(inplace = True)

In [7]:
data.head(2)

,index,id,author,source,content,feature,title,url
0,0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm


### 2 分词

In [8]:
import jieba as jb

- 停用词


In [9]:
with open('chinese_stopwords.txt','r',encoding = 'utf-8') as file:
    stopword_list = [line.strip() for line in file.readlines()]

In [10]:
stopword_list[:3]

['，', '的', '。']

In [11]:
stopword_list[1] #== ''

'的'

In [12]:
none_sense_str = ['\n',' ','\t','\r']
stopword_list += none_sense_str

In [13]:
def sentence_split(text):
    print(text)
    text = text.replace(' ', '').replace('\n', '').replace('\r', '')
    text = jb.cut(text)
    text = [i for i in text if i not in stopword_list]
    return ' '.join(text)
    

In [14]:
sentence_split('佟丽娅真好看，\n陈思诚是个狗东西')

Building prefix dict from the default dictionary ...


佟丽娅真好看，
陈思诚是个狗东西


Dumping model to file cache /var/folders/9j/lcz7xrtn1wz6_8x9m2qg2zjc0000gn/T/jieba.cache
Loading model cost 0.736 seconds.
Prefix dict has been built successfully.


'佟丽娅 真 好看 陈思诚 狗东西'

In [15]:
import os
import pickle

In [16]:
data.head()

,index,id,author,source,content,feature,title,url
0,0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [17]:
if os.path.exists('./have_cut.pkl'):
    with open('./have_cut.pkl', 'rb') as file:
        have_cut = pickle.load(file)

else:
    have_cut = data['content'].apply(sentence_split)
    # 保存到本地，方便下次读取
    with open('./have_cut.pkl', 'wb') as file:
        pickle.dump(have_cut, file)

In [18]:
have_cut

0        此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测...
1        骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 ...
2        此前 一加 3T 搭载 3400mAh 电池 DashCharge 快充 规格 5V 4A ...
3        这是 18 葡萄牙 中部 佩德罗 冈 地区 拍摄 森林 大火 烧毁 汽车 新华社 记者 张立 云摄
4        原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 裸身 奔走 … … @ 深圳 交警...
                               ...                        
87047    新华社 照片 多伦多 2017 \ n 体育 冰球 国家 女子 冰球队 海外 选秀 多伦多 ...
87048    新华社 兰州 日电 王衡 徐丹 记者 甘肃省 交通运输 厅 获悉 甘肃 近日 集中 开建 高...
87049    \ n \ n2017 29 法国巴黎 郊外 凡尔赛宫 法国 总统 马克 龙 出席 新闻 发...
87050    \ n \ n2017 25 美国 马萨诸塞州 剑桥市 哈佛大学 毕业生 毕业典礼 欢呼 新...
87051    　 　 新华社 德国 杜塞尔多夫 ６ ６ 日电 题 乒乓 女球迷 　 \ n 　 　 新华社...
Name: content, Length: 87052, dtype: object

In [19]:
type(have_cut)

pandas.core.series.Series

## 生成tf_idf矩阵

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.model_selection import train_test_split 

In [21]:
label = np.array([1 if '新华社' in i else 0 for i in data.source])

In [22]:
sum(label)

78674

In [23]:
data.shape

(87052, 8)

In [24]:
### 87052篇文章中抄袭的文章有78674篇，占比还挺高

In [25]:
X_train,X_test,y_train,y_test = train_test_split(have_cut,label,test_size = 0.25)

In [26]:
countvectorizer = CountVectorizer(encoding='gb18030', min_df=0.01)

In [27]:
countvector_train = countvectorizer.fit_transform(X_train)

In [28]:
countvector_train.shape

(65289, 1399)

In [29]:
countvector_train_array = countvector_train.toarray()

In [30]:
countvector_test_array = countvectorizer.transform(X_test).toarray()

In [31]:
### 转化为tf-idf的矩阵
tfidftransformer = TfidfTransformer()
tfidf_train = tfidftransformer.fit_transform(countvector_train_array).toarray()
tfidf_test = tfidftransformer.transform(countvector_test_array).toarray()

In [32]:
X_train.shape

(65289,)

In [33]:
tfidf_train.shape

(65289, 1399)

In [34]:
tfidf_train[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 进行分类预测

In [35]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [36]:
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME",
                         n_estimators=100, learning_rate=0.5)

In [37]:
bdt.fit(tfidf_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(max_depth=2,
                                                         min_samples_leaf=5,
                                                         min_samples_split=20),
                   learning_rate=0.5, n_estimators=100)

In [38]:
from sklearn.metrics import classification_report

In [39]:
y_train_pred = bdt.predict(tfidf_train)
y_test_pred = bdt.predict(tfidf_test)

In [40]:
print(classification_report(y_test, y_test_pred, labels=[0, 1], target_names=['others', '新华社']))

              precision    recall  f1-score   support

      others       0.92      0.95      0.93      1983
         新华社       0.99      0.99      0.99     19780

    accuracy                           0.99     21763
   macro avg       0.96      0.97      0.96     21763
weighted avg       0.99      0.99      0.99     21763



In [41]:
print(classification_report(y_train, y_train_pred, labels=[0, 1], target_names=['others', '新华']))

              precision    recall  f1-score   support

      others       0.93      0.94      0.94      6395
          新华       0.99      0.99      0.99     58894

    accuracy                           0.99     65289
   macro avg       0.96      0.97      0.96     65289
weighted avg       0.99      0.99      0.99     65289



### 应用模型进行预测  
- 也就是模型判断文章为新华社的，但是实际不是新华的，这种应该交由人工做重点审查

In [42]:
type(tfidf_test)

numpy.ndarray

In [43]:
tfidf_all = np.vstack((tfidf_test,tfidf_train))
label_all = np.hstack((y_test,y_train))

In [44]:
label_predict = bdt.predict(tfidf_all)

In [45]:
index1 = list((label_all) != 1 & (label_predict ==1))

In [46]:
type(index1)

list

In [47]:
X_all = X_test.append(X_train)

In [48]:
X_all

24432    新华社 照片 深州 河北 2017 13 \ n 河北 深州 非遗 文化 进校园 \ n4 ...
24856    新华社 照片 外代 2017 14 \ n 外代 二线 足球 欧罗巴 联赛 里昂 战胜 贝西...
60972    新华社 照片 外代 2017 16 \ n 外代 二线 无名 湖上 海鸥 \ n5 海鸥 俄...
84284    　 　 我社 内部 分工 调整 现将 发稿 值班 联系电话 公布 如下 \ n 国内部 负责...
71954    新华社 照片 长春 2017 25 \ n 吉林 外国人 感受 端午 文化 \ n5 25 ...
                               ...                        
34701    新华社 照片 重庆 2017 22 \ n 第十七届 中国 青少年 机器人 竞赛 重庆 赛区...
43751    　 　 海上 军演 消停 　 半岛 局势 惹人忧 \ n 　 　 　 　 杜鹃 \ n 　 ...
65462    新华社 照片 戛纳 法国 2017 19 \ n 国际 影片 玉子 主创 亮相 戛纳 \ n...
8420     佛山 南海 西樵山 听音 湖畔 崛起 一座 岭南 文旅 小镇 13 上午 凝聚 动能 布局 ...
42480    新华社 照片 外代 2017 29 \ n 外代 二线 西班牙 斗牛士 英姿 \ n4 28...
Name: content, Length: 87052, dtype: object

In [49]:
doc_copy_cut = X_all.loc[index1]

In [50]:
doc_copy_cut

52359                                              \ n \ n
85837    \ n \ n \ n \ n2017 高考 即将来临 今年 称为 高考 元年 2014 国...
2290     点击 图片 进入 一页 春节 期间 廖俊波 左一 政和县 松源村 慰问 群众 2015 日摄...
5675     昨天 故宫 老农 专门 开 场 追思会 　 　 新华社 电 记者 双瑞 王 林园 　 昨日 ...
4508     凤凰 WEMONEY 讯 中国 金融 改革 发展 不仅 全球 视野 更要 经济 视野 不能 ...
                               ...                        
7599     图文 鱼水情深 满 香江 ― ― 驻 香港 部队 探索 构建 特殊 条件 新型 军政 军民关...
6408     21 首轮 美 外交 安全 对话 美国 华盛顿 举行 这是 一场 建设性 富有成果 对话 国...
59011            \ nhttp h5 zmiti com public beltRoad1 \ n
7590     北京 时间 21 凌晨 摩根士丹利 资本 国际 公司 MSCI 宣布 A股 纳入 MSCI ...
6316     新华社 济南 22 日电 记者 萧海川 记者 国家 超级 计算 济南 中心 获悉 我国 下一...
Name: content, Length: 1096, dtype: object

In [52]:
doc_copy = data.loc[doc_copy_cut.index]

In [53]:
for i in doc_copy.source:
    if '新华社' in i:
        print(i)

新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社@
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社

In [54]:
for i in range(len(label)):
    if label[i] == 1 and '新华社' not in data['source'].loc[i]:
        print(i)

In [55]:
data.shape

(87052, 8)

In [56]:
5134 in data.index

True